## INFO834 TP2     


In [88]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq

In [89]:

file_path_1 = './data/academies_virgule.csv'

file_path_2 = './data/villes_virgule.csv'

dfAcademies = pd.read_csv(file_path_1)
dfVilles = pd.read_csv(file_path_2)

print(dfAcademies)
print(dfVilles)


                         academie                              wikipedia  \
0                Académie de Lyon        fr:Académie de Lyon (éducation)   
1               Académie d'Amiens       fr:Académie d'Amiens (éducation)   
2    Académie de Clermont-Ferrand        fr:Académie de Clermont-Ferrand   
3        Académie d'Aix-Marseille            fr:Académie d'Aix-Marseille   
4        Académie d'Aix-Marseille            fr:Académie d'Aix-Marseille   
..                            ...                                    ...   
96         Académie de Versailles  fr:Académie de Versailles (éducation)   
97      Académie de la Guadeloupe           fr:Académie de la Guadeloupe   
98      Académie de la Martinique           fr:Académie de la Martinique   
99             Académie de Guyane               fr:Académie de la Guyane   
100        Académie de la Réunion              fr:Académie de La Réunion   

    vacances  dep              departement                       region  
0     Zone A 

### Question 1) 
 Quatre fonctions qui permettent de convertir les données données en paramètre d'un format vers l'autre
(en lecture et écriture) : dataframe < - > table, table < - > fichier parquet. Tester sur les données des villes
et académies.

In [90]:
tableAcademies =  pa.Table.from_pandas(dfAcademies)
tableVilles =  pa.Table.from_pandas(dfVilles)

In [91]:
print(tableAcademies)
print(tableVilles)

pyarrow.Table
academie: string
wikipedia: string
vacances: string
dep: string
departement: string
region: string
----
academie: [["Académie de Lyon","Académie d'Amiens","Académie de Clermont-Ferrand","Académie d'Aix-Marseille","Académie d'Aix-Marseille",...,"Académie de Versailles","Académie de la Guadeloupe","Académie de la Martinique","Académie de Guyane","Académie de la Réunion"]]
wikipedia: [["fr:Académie de Lyon (éducation)","fr:Académie d'Amiens (éducation)","fr:Académie de Clermont-Ferrand","fr:Académie d'Aix-Marseille","fr:Académie d'Aix-Marseille",...,"fr:Académie de Versailles (éducation)","fr:Académie de la Guadeloupe","fr:Académie de la Martinique","fr:Académie de la Guyane","fr:Académie de La Réunion"]]
vacances: [["Zone A","Zone B","Zone A","Zone B","Zone B",...,"Zone C",null,null,null,null]]
dep: [["01","02","03","04","05",...,"95","971","972","973","974"]]
departement: [["Ain","Aisne","Allier","Alpes-de-Haute-Provence","Hautes-Alpes",...,"Val-d'Oise","Guadeloupe","Marti

In [92]:
pq.write_table(tableAcademies, "./parquet/question1.parquet")


### Question 2) 
 Une fonction qui permet d’afficher le schéma d'une table. Tester sur les données villes et académies.


In [93]:
print(tableAcademies.schema)
print(tableVilles.schema)

academie: string
wikipedia: string
vacances: string
dep: string
departement: string
region: string
-- schema metadata --
pandas: '{"index_columns": [{"kind": "range", "name": null, "start": 0, "' + 951
dep: string
nom: string
cp: string
nb_hab_2010: int64
nb_hab_1999: int64
nb_hab_2012: int64
dens: int64
surf: double
long: double
lat: double
alt_min: double
alt_max: double
-- schema metadata --
pandas: '{"index_columns": [{"kind": "range", "name": null, "start": 0, "' + 1596


### Question 3) 
 Une fonction qui renvoie la colonne col d'une table donnée en paramètre (fonction column() ou select()).

In [94]:
def column(table, columnName):
    return pd.DataFrame(table[columnName])

column(tableAcademies, "wikipedia")
    

,0
0,fr:Académie de Lyon (éducation)
1,fr:Académie d'Amiens (éducation)
2,fr:Académie de Clermont-Ferrand
3,fr:Académie d'Aix-Marseille
4,fr:Académie d'Aix-Marseille
...,...
96,fr:Académie de Versailles (éducation)
97,fr:Académie de la Guadeloupe
98,fr:Académie de la Martinique
99,fr:Académie de la Guyane


### Question 4) 
En utilisant les fonctions compute de pyarrow, des fonctions qui renvoient des statistiques sur une
colonne de table : count, count_distinct, sum, min, max. Tester sur des colonnes de la table des villes.

### Question 5)
Filtrage et tri : des fonctions pour sélectionner les informations sur une ville (e.g. Annecy) et pour
sélectionner les informations sur un département (e.g. Haute-Savoie) par ordre alphabétique des villes
(fonction filter et sort).

### Question 6)
Calculs sur plusieurs colonnes et agrégats : calcul du nombre moyen d'habitants en 2012 ; calcul du
nombre moyen d'habitants par département ; afficher le résultat pour le département 74 (fonction
TableGroupBy).

### Question 7)
Opérations ensemblistes jointures : afficher les zones de vacances des villes ; les villes de la zone de
vacances A ; les départements des zones de vacances A et B ; le nombre de villes par académie ;

### Question 8)
Affichage avec matplotlib : e.g. histogramme de la distribution du nombre de villes par académie.